In [ ]:
import lalgwpe as gwpe
import numpy as np
import matplotlib.pyplot as plt
import lal
import pickle
import os

In [ ]:
# set all parameters for the waveform

# temporal duration of signal, seconds
T = 128
# sampling rate, Hertz
Fs = 4096

# strings corresponding to used detectors, code only supports LIGO Hanford/Livingston and Virgo now
detstrings = ['H1', 'L1', 'V1']

# instantiate parameter dictionary
params = {}

### intrinsic parameters of the black holes
### masses [solar masses]
params['mass_1'] = 1.4 # m1 > m2
params['mass_2'] = 1.2 # code swaps masses if m2 > m1, but don't do this, it'll be confusing
### spins [normalized, Kerr bound requires that magnitudes of these are <1]
params['chi_1x'] = 0
params['chi_1y'] = 0
params['chi_1z'] = 0.5
params['chi_2x'] = 0
params['chi_2y'] = 0
params['chi_2z'] = 0.8
### dimensionless tidal deformabilities, these are zero for black holes, bc of the no hair theorem
params['Lambda1'] = 0.0
params['Lambda2'] = 0.0

### extrinsic parameters
params['distance'] = 40*lal.PC_SI*1e6 # luminosity distance [m] [Mpc when multiplied by lal.PC_SI*1e6]
params['phi_ref'] = 1.5 # phase offset at reference frequency [rad]
params['inclination'] = 1.0 # inclination angle of orbital angular momentum of binary
params['ra'] = 3.4416 # right ascension (azimuthal angle of source wrt earth 0<ra<2pi)
params['dec'] = -0.408084 # declination (polar angle of source wrt earth 0<dec<pi)
params['psi'] = 1.25 # roll angle of orbital angular momentum of binary
params['tgps'] = 1187008882.43 + 15*60 # gps time of merger (15 mins after time of GW170817)
params['tcoarse'] = T/2 # time of merger in time series (0<tcoarse<T)
params['tc'] = 0 # fine offset time of merger from tcoarse

### frequencies used for the waveform conventions
params['f_ref'] = 50 # reference frequency (unphysical convention that sets the phase offset) [Hz]
params['f_min'] = 30 # Minimum frequency for model, when the model "turns on", keep this reasonably high or the waveform will overlap onto itself in multiple wraps
params['f_max'] = Fs/2 # Half of the sampling frequency is the Nyquist frequency, which is the highest frequency stored by the frequency array (why is this?)

In [ ]:
### compute time domain strain
h_tf2 = gwpe.compute_strain_td(params, T, Fs, "TaylorF2")
### corresponding time array
t = np.linspace(-T/2, T/2-1/Fs, T*Fs)

In [ ]:
%matplotlib inline
plt.figure(figsize = (20,6))
plt.plot(t, h_tf2[0], label = 'LIGO Hanford')
plt.plot(t, h_tf2[1], label = 'LIGO Livingston')
plt.plot(t, h_tf2[2], label = 'Virgo')
plt.xlabel('time after merger (s)')
plt.ylabel('strain')
plt.xlim(-0.2, 0.03)
plt.axvline(params['tcoarse'], label = 'merger time in reference detector (Livingston)')
plt.legend()
plt.show()

In [ ]:
### compute frequency domain strain
hf_tf2 = gwpe.compute_strain_fd(params, T, Fs, "TaylorF2")
### corresponding frequency array
f = np.linspace(0, Fs/2, T*Fs//2+1)

In [ ]:
%matplotlib inline
plt.figure(figsize = (20,6))
plt.plot(f, abs(hf_tf2[0]), label = 'LIGO Hanford')
plt.plot(f, abs(hf_tf2[1]), label = 'LIGO Livingston')
plt.plot(f, abs(hf_tf2[2]), label = 'Virgo')
plt.xlabel('Frequency (Hz)')
plt.ylabel('strain')
plt.xlim(0, 100)
plt.legend()
plt.show()

In [ ]:
# Getting data, psd, etc.
T_long = 8*T
tgps = params['tgps']

# #=============================================================
# # This block pulls the data from gw-openscience.org, runs welch 
# # and stores the object in the pkl file, which taken a minute 
# # or so. Comment this whole block out after this file is created
# # to avoid this extra runtime.
# d = gwpe.data(T, Fs, T_long, tgps)
# if not os.path.exists('pkl'):
#     os.mkdir('pkl')
# afile = open('pkl/15afterGW170817_data.pkl', 'wb')
# pickle.dump(d, afile)
# afile.close()
# #=============================================================

# reload data from file, only this needs to be run to get the data
# object once the pkl file is generated
file = open('pkl/15afterGW170817_data.pkl', 'rb')
d = pickle.load(file)
file.close()

# get data and psd from the data object
psd = d.psd
fs = d.fs
df = d.strain_fd
dt = d.strain_td

In [ ]:
# compute time and frequency domain strains for IMRPhenomXPHM model
h_xphm = gwpe.compute_strain_td(params, T, Fs, "IMRPhenomXPHM")
hf_xphm = gwpe.compute_strain_fd(params, T, Fs, "IMRPhenomXPHM")

In [ ]:
%matplotlib inline
plt.figure(figsize = (20,6))
plt.plot(t, h_tf2[0], label = 'TaylorF2')
plt.plot(t, h_xphm[0], label = 'IMRPhenomXPHM, unaligned')
plt.xlabel('time after merger (s)')
plt.ylabel('strain')
plt.xlim(-0.2, 0.03)
plt.axvline(params['tcoarse'], label = 'merger time in reference detector (Livingston)')
plt.legend()
plt.show()

In [ ]:
cos_unaligned = gwpe.overlap_cosine(hf_xphm, hf_tf2, f, psd, T, params['f_min'], params['f_max'])
print(cos_unaligned)

In [ ]:
h_xphm_matched = gwpe.match_phase_td(h_xphm, h_tf2, f, psd, T, Fs, params['f_min'], params['f_max'])
hf_xphm_matched = gwpe.td_to_fd(h_xphm_matched, T, Fs)
cos_aligned = gwpe.overlap_cosine(hf_xphm_matched, hf_tf2, f, psd, T, params['f_min'], params['f_max'])
print(cos_aligned)

In [ ]:
%matplotlib inline
plt.figure(figsize = (20,6))
plt.plot(t, h_tf2[0], label = 'TaylorF2')
plt.plot(t, h_xphm_matched[0], label = 'IMRPhenomXPHM, aligned')
plt.xlabel('time after merger (s)')
plt.ylabel('strain')
plt.xlim(-0.3, 0.01)
plt.axvline(params['tcoarse'], label = 'merger time in reference detector (Livingston)')
plt.legend()
plt.show()

In [ ]:
#Goal: Compute and plot h+, hx, and (frequency) derivatives in frequency domain
# def hpc(params, f_seq, modelstring): 
#     """
#     Computes the plus and cross polarizations for the dict params and model, at the frequencies in f_seq by calling lal for the polarizations directly.
#     params: parameter dictionary
#     f_seq: REAL8Sequence LAL object that contains a frequency array
#     modelstring: strying specifying which model to use, some possible values include "TaylorF2", IMRPhenomD", "IMRPhenomXPHM"
#     Returns (h+ array, hx array)

#"""
#Use IMRPhenomD_NRTidalv2
# f_seq = lal.CreateREAL8Sequence(len(fs)) #creates an object with two properties: length, data (np array)
#     f_seq.data = fs

In [ ]:
modelstring = "TaylorF2"
#Want "IMRPhenomD_NRTidalv2" or something that takes into account tidal deformability as a parameter

def finite_differencing(f, df):
    """
    f - Frequency sequence at which to evaluate derivative
    df is the finite differencing hyperparameter
    """
    # f_seq is inps
    # REAL8Sequence needed for LAL
    f_seq = lal.CreateREAL8Sequence(len(f))
    f_seq.data = f
    #df_seq is offset inps
    df_seq = lal.CreateREAL8Sequence(len(f))
    df_seq.data = f+df
    
    y2 = gwpe.hpc(params, df_seq, modelstring)
    y1 = gwpe.hpc(params, f_seq, modelstring)

    h_plus = y1[0]
    h_cross = y1[1]

    dh_plus = (y2[0]-y1[0])/df
    dh_cross = (y2[1]-y1[1])/df

    return (h_plus, dh_plus, h_cross, dh_cross)

In [ ]:
# df is used for finite differencing, h'(f) = [h(f + df) - h(f)] / df
# delta_f: frequency difference between h'(f) array entries
df = 1e-9
fmin = 30
fmax = 100
delta_f = 0.01
f = np.arange(fmin, fmax, delta_f)
data = finite_differencing(f, df)

In [ ]:
data_real = np.real(data)
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

# Plot on the first subplot
axs[0][0].plot(f, data_real[0], color='blue', label='h+')
axs[0][0].set_title('h_plus (real)')
axs[0][0].legend()

# Plot on the second subplot
axs[0][1].plot(f, data_real[2], color='red', label='hx')
axs[0][1].set_title('h_cross (real)')
axs[0][1].legend()

# Plot on the third subplot
axs[1][0].plot(f, data_real[1], color='blue', label='d_h+')
axs[1][0].set_title('h_plus derivative (real)')
axs[1][0].legend()

# Plot on the fourth subplot
axs[1][1].plot(f, data_real[3], color='red', label='d_h+')
axs[1][1].set_title('h_cross derivative (real)')
axs[1][1].legend()

# Adjust layout to prevent clipping of titles
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
data_imag = np.imag(data)
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

# Plot on the first subplot
axs[0][0].plot(f, data_imag[0], color='blue', label='h+')
axs[0][0].set_title('h_plus (imag)')
axs[0][0].legend()

# Plot on the second subplot
axs[0][1].plot(f, data_imag[2], color='red', label='hx')
axs[0][1].set_title('h_cross (imag)')
axs[0][1].legend()

# Plot on the third subplot
axs[1][0].plot(f, data_imag[1], color='blue', label='d_h+')
axs[1][0].set_title('h_plus derivative (imag)')
axs[1][0].legend()

# Plot on the fourth subplot
axs[1][1].plot(f, data_imag[3], color='red', label='d_h+')
axs[1][1].set_title('h_cross derivative (imag)')
axs[1][1].legend()

# Adjust layout to prevent clipping of titles
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
#Make accuracy of finite differencing at f=50Hz plot
f = np.array([50])
deltax_arr = np.geomspace(1e-20, 1, 1000)
dh_plus_at50 = np.zeros(1000)
dh_cross_at50 = np.zeros(1000)
for i, df in enumerate(deltax_arr):
    data = np.real(finite_differencing(f, df))
    dh_plus_at50[i] = data[1][0]
    dh_cross_at50[i] = data[3][0]

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 24))

axs[0].semilogx(deltax_arr, dh_plus_at50)
axs[0].set_title("Finite Differencing Hyperparameter Test (h+)")
axs[0].set_xlabel("df")
axs[0].set_ylabel("Evaluated h+'(f)")

axs[1].semilogx(deltax_arr, dh_cross_at50)
plt.title("Finite Differencing Hyperparameter Test (hx)")
plt.xlabel("df")
plt.ylabel("Evaluated hx'(f)")
plt.show()